<a href="https://colab.research.google.com/github/FuturisticLover123/learning/blob/main/ifortex_test/learing_model_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.7 MB/s eta 0:00:00


In [40]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

Загрузка датасета в папку content

In [5]:
!wget https://github.com/FuturisticLover123/learning/raw/main/ifortex_test/student_exam_data.zip
!unzip student_exam_data.zip -d /content

--2025-04-26 11:09:23--  https://github.com/FuturisticLover123/learning/raw/main/ifortex_test/student_exam_data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FuturisticLover123/learning/main/ifortex_test/student_exam_data.zip [following]
--2025-04-26 11:09:23--  https://raw.githubusercontent.com/FuturisticLover123/learning/main/ifortex_test/student_exam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2444 (2.4K) [application/zip]
Saving to: ‘student_exam_data.zip.1’

student_exam_data.z 100%[===================>]   2.39K  --.-KB/s    in 0s      

2025-04-26 11:09:23 (33.0 MB/s) - 

In [7]:
data = pd.read_csv("/content/student_exam_data.csv")

Посмотрим данные и обновные характеристики

In [8]:
data.describe()

,Контрольная 1,Контрольная 2,Контрольная 3
count,300.000000,300.000000,300.000000
mean,7.130000,6.936667,6.940000
std,2.003283,2.009839,2.028987
min,4.000000,4.000000,4.000000
25%,5.000000,5.000000,5.000000
50%,7.000000,7.000000,7.000000
75%,9.000000,9.000000,9.000000
max,10.000000,10.000000,10.000000


In [9]:
data.isna().sum()

,0
Контрольная 1,0
Контрольная 2,0
Контрольная 3,0
Сон накануне,0
Настроение,0
Энергетиков накануне,0
Посещаемость занятий,0
Время подготовки,0
Сдал,0


В данных нет пропусков это очень хорошо

In [10]:
data.head()

,Контрольная 1,Контрольная 2,Контрольная 3,Сон накануне,Настроение,Энергетиков накануне,Посещаемость занятий,Время подготовки,Сдал
0,4,4,9,Нет,Хорошее,4+,Низкая,За неделю,Нет
1,9,9,6,Да,Плохое,1,Средняя,Последний час,Да
2,8,6,9,Да,Хорошее,2-3,Средняя,Последний час,Да
3,8,4,9,Да,Хорошее,1,Высокая,Последняя ночь,Да
4,9,10,4,Нет,Нормальное,1,Низкая,Последний час,Да


Заметим что в данных много категориальных признаков. Данных не много поэтому нейронки особого смысла обучать нет + много категориальных признаков поэтому основная модель скорее всего будет градиентным бустингом над деревьями.  

Но для построения бэйслайнов которые не могут работаь с категориальными признаками и некоторой предобработки декодируем их. C помощью one hot encoding

In [11]:
x, y = data.drop(columns = ["Сдал"]), data["Сдал"]

Все столбцы кроме оценок считаем категориальными

In [12]:
cat_columns = x.iloc[:, 3:].columns
df_encoded = pd.get_dummies(x, columns=x.iloc[:, 3:].columns).astype(int)

С помощью Изоляционного леса предобработает анамалии. Порог подбирался вручную

In [13]:
iso = IsolationForest(random_state=0, contamination=0.01)
clean = iso.fit_predict(df_encoded, y)

X_cleaned = x[clean != -1]
X_encoded_clean = df_encoded[clean != -1]

y_cleaned = y[clean != -1]

Из понимания домена делаем вывод что наиболее важные факторы это оценки и посещаемость. Поэтому делаем вывод что это скорее всего действительно аномалии

In [14]:
data.iloc[clean == -1]

,Контрольная 1,Контрольная 2,Контрольная 3,Сон накануне,Настроение,Энергетиков накануне,Посещаемость занятий,Время подготовки,Сдал
7,8,6,10,Нет,Хорошее,4+,Средняя,За несколько дней,Нет
32,9,10,10,Нет,Плохое,4+,Средняя,За неделю,Нет
99,4,10,9,Нет,Плохое,4+,Средняя,За неделю,Нет


Попробуем добавить новых фичей

Из понимания домена можно попробовать сделать вывод что наибольший вклад дают именно оценки. Поэтому попробуем добавить несколько статискик этих данных

In [15]:
X_cleaned.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)
X_encoded_clean.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)

<ipython-input-15-16e3bd562b50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cleaned.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)
<ipython-input-15-16e3bd562b50>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_encoded_clean.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)


Добавили среднее из оценок

In [16]:
X_encoded_clean.loc[:, "Min"] = X_encoded_clean[X_encoded_clean.columns[:3]].min(axis=1)

X_encoded_clean = X_encoded_clean.assign(
    Min=X_encoded_clean[X_encoded_clean.columns[:3]].min(axis=1)
)

<ipython-input-16-0ba08c6f1b68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_encoded_clean.loc[:, "Min"] = X_encoded_clean[X_encoded_clean.columns[:3]].min(axis=1)


Добавили минимальную оценку

Попробуем обучить несколько простых моделей

In [17]:
Xen_train, Xen_test, yen_train, yen_test = train_test_split(X_encoded_clean, y_cleaned)

In [18]:
lr = LogisticRegression()
print(cross_val_score(lr, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.8647727272727274


In [19]:
knn = KNeighborsClassifier(n_neighbors=3)
print(cross_val_score(knn, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.7886363636363636


In [20]:
nb = GaussianNB()
print(cross_val_score(nb, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.8473484848484848


К сожалению простые модели не дали хорошей точности. Их можно былобы потюнить и попытаться поднять точность на пару процентов. Но как мы уже говорили датасет имеет много категориальных фич поэтому имеет смысл хорошо обучить catboost

In [21]:

X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned)
cat_features =  [3,4,5,6,7]

In [22]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 10, 50),
    }

    kf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
        val_pool = Pool(X_val, y_val, cat_features=cat_features)

        model = CatBoostClassifier(**params, verbose=0)
        model.fit(train_pool, eval_set=val_pool)

        y_pred = model.predict_proba(X_val)[:, 1]
        scores.append(log_loss(y_val, y_pred))

    return np.mean(scores)

Для подбора гиперпараметров используем optuna с разбиением на 20 фолдов

In [23]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, timeout=3600)

[I 2025-04-26 11:11:19,053] A new study created in memory with name: no-name-6d57441e-b7dc-4b88-b4aa-f907a283821c
[I 2025-04-26 11:11:27,064] Trial 0 finished with value: 0.18341305357350562 and parameters: {'iterations': 270, 'learning_rate': 0.08217267564630482, 'depth': 10, 'l2_leaf_reg': 0.010159554193949477, 'random_strength': 6.858268893101574, 'bagging_temperature': 0.6135080492749914, 'min_data_in_leaf': 47, 'grow_policy': 'Depthwise', 'early_stopping_rounds': 11}. Best is trial 0 with value: 0.18341305357350562.
[I 2025-04-26 11:11:34,241] Trial 1 finished with value: 0.1494000332969112 and parameters: {'iterations': 422, 'learning_rate': 0.153239686642538, 'depth': 10, 'l2_leaf_reg': 0.0015560361286817595, 'random_strength': 8.826924666498826, 'bagging_temperature': 0.527570270505195, 'min_data_in_leaf': 4, 'grow_policy': 'Lossguide', 'early_stopping_rounds': 18}. Best is trial 1 with value: 0.1494000332969112.
[I 2025-04-26 11:11:45,758] Trial 2 finished with value: 0.252617

In [24]:
best_params = study.best_params
print("Best params:", best_params)

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train)

Best params: {'iterations': 915, 'learning_rate': 0.2572589843146301, 'depth': 8, 'l2_leaf_reg': 1.971016556972674, 'random_strength': 2.733052261327682, 'bagging_temperature': 0.9925379070570429, 'min_data_in_leaf': 72, 'grow_policy': 'SymmetricTree', 'early_stopping_rounds': 49}


Смотрим лучшие параметры и добавляем валиданионнцю выборку

In [25]:
final_model = CatBoostClassifier(**best_params,  verbose=100)
final_model.fit(
    Pool(X_train, y_train, cat_features=cat_features),
    eval_set=Pool(X_val, y_val, cat_features=cat_features),
    early_stopping_rounds=50
)

0:	learn: 0.5314309	test: 0.5636950	best: 0.5636950 (0)	total: 6.45ms	remaining: 5.89s
100:	learn: 0.0039100	test: 0.1649016	best: 0.1643297 (99)	total: 701ms	remaining: 5.65s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1643297055
bestIteration = 99

Shrink model to first 100 iterations.


Благодаря early stopping удалось урезать количество деревьев и помогает избегать переобучения

In [26]:
print(accuracy_score(final_model.predict(X_cleaned), y_cleaned))
print(accuracy_score(final_model.predict(X_test), y_test))

0.9730639730639731
0.96


Основная метрика на train и test высокие. Нет переобчучения

In [41]:
class FeatureAdder(BaseEstimator, TransformerMixin):
  def __init__(self):
      pass

  def fit(self, X, y=None):
      return self

  def transform(self, X: pd.DataFrame):
      X = X.copy()
      if isinstance(X, pd.DataFrame):
        cols = X.columns[:3]

        X['Mean'] = X[cols].mean(axis=1, skipna=True)
        X['Min'] = X[cols].min(axis=1, skipna=True)
        return X

pipeline = Pipeline([
    ('feature_adder', FeatureAdder()),
    ('classifier', final_model)
])

Обьединим преобразования в pipeline для удобства использования

In [46]:
print(accuracy_score(pipeline.predict(X_cleaned), y_cleaned))
print(accuracy_score(pipeline.predict(X_test), y_test))

0.9730639730639731
0.96


Все работает

In [48]:
# from joblib import dump, load

# dump(pipeline, 'pipeline.joblib')

['pipeline.joblib']

Сохранение модели если нужно. Копия модели которая используется в файле интерфейса хранится в на моем github. Скачивать эту модель не обязательно.